In [1]:
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import vbfprocessor
import awkward as ak
import uproot

import time

from distributed import Client
from lpcjobqueue import LPCCondorCluster

from dask.distributed import performance_report
from dask_jobqueue import HTCondorCluster, SLURMCluster

In [2]:
cluster = LPCCondorCluster()
cluster.adapt(minimum=1, maximum=50)

cluster = LPCCondorCluster(
    job_script_prologue=[
        "export DASK_DISTRIBUTED__WORKER__PROFILE__INTERVAL=1d",
        "export DASK_DISTRIBUTED__WORKER__PROFILE__CYCLE=2d",
    ],
    memory="6GB",
    ship_env=True,
    shared_temp_directory="/tmp",
    transfer_input_files=["vbfprocessor.py"],
    image="coffeateam/coffea-dask:0.7.20-fastjet-3.4.0.1-g4cab023"
)

/opt/conda/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34513 instead
  warnings.warn(


In [3]:
fileset = {
    "VBF_SMEFTsim_topU3l_NP1": [
        "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_NP1/VBF_SMEFTsim_topU3l_NP1/230526_132704/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(0,200)
    ]
}

In [4]:
tstart = time.time()
with Client(cluster) as client:
    
    with performance_report(filename="dask-report.html"):

        uproot.open.defaults["xrootd_handler"] = uproot.source.xrootd.MultithreadedXRootDSource

        p = vbfprocessor.VBFProcessor(isMC=True)
        args = {'savemetrics':True, 'schema':NanoAODSchema}

        output = processor.run_uproot_job(
            fileset,
            treename="Events",
            processor_instance=p,
            executor=processor.dask_executor,
            executor_args={
                "client": client,
                "skipbadfiles": 1,
                "schema": processor.NanoAODSchema,
                "treereduction": 2,
            },
            chunksize=50000,
        )

        outfile = "coffea/VBF_SMEFTsim_topU3l_NP1.coffea"
        util.save(output, outfile)
        print("saved " + outfile)
        print(datetime.now())

[                                        ] | 0% Completed | 11min 44.2s

KeyboardInterrupt: 